## Notebook for exploratory analysis of Fetal Gut Stem cells scRNA-Seq data using `scVI` 

- **Developed by:** Anna Maguza
- **Place:** Wuerzburg Institute for System Immunology
- **Date:** 16th November 2023

### Load required modules

In [1]:
import sys
import scvi
import torch
import anndata
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
import seaborn as sns

import numpy as np
import scipy as sp
import pandas as pd
import scanpy as sc
import numpy.random as random


from umap import UMAP
import warnings; warnings.simplefilter('ignore')

import matplotlib.pyplot as plt

/home/amaguza/miniforge3/envs/scvi_env/lib/python3.9/site-packages/scvi/_settings.py:63: UserWarning: Since v1.0.0, scvi-tools no longer uses a random seed by default. Run `scvi.settings.seed = 0` to reproduce results from previous versions.
  self.seed = seed
/home/amaguza/miniforge3/envs/scvi_env/lib/python3.9/site-packages/scvi/_settings.py:70: UserWarning: Setting `dl_pin_memory_gpu_training` is deprecated in v1.0 and will be removed in v1.1. Please pass in `pin_memory` to the data loaders instead.
  self.dl_pin_memory_gpu_training = (
/home/amaguza/miniforge3/envs/scvi_env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
%matplotlib inline
matplotlib.rcParams["pdf.fonttype"] = 42
matplotlib.rcParams["ps.fonttype"] = 42

In [3]:
torch.cuda.is_available()

True

In [4]:
torch.set_float32_matmul_precision('medium')

In [5]:
sc.settings.verbosity = 3
sc.logging.print_versions()
sc.settings.set_figure_params(dpi = 180, color_map = 'magma_r', dpi_save = 300, vector_friendly = True, format = 'svg')

-----
anndata     0.10.3
scanpy      1.9.6
-----
PIL                         10.1.0
absl                        NA
aiohttp                     3.8.6
aiosignal                   1.3.1
annotated_types             0.6.0
anyio                       NA
arrow                       1.3.0
asttokens                   NA
async_timeout               4.0.3
attr                        23.1.0
attrs                       23.1.0
babel                       2.13.1
backoff                     2.2.1
brotli                      1.1.0
bs4                         4.12.2
certifi                     2023.07.22
cffi                        1.16.0
charset_normalizer          3.3.2
chex                        0.1.7
click                       8.1.7
colorama                    0.4.6
comm                        0.1.4
contextlib2                 NA
croniter                    NA
cycler                      0.12.1
cython_runtime              NA
dateutil                    2.8.2
debugpy                     1.8.0
decor

In [6]:
def X_is_raw(adata):
    return np.array_equal(adata.X.sum(axis=0).astype(int), adata.X.sum(axis=0))

### Read in datasets

- Read in formatted object

In [9]:
adata = sc.read_h5ad('/home/amaguza/data/Processed_data/Gut_data/Stem_cells/Fetal_cells.h5ad')
adata

AnnData object with n_obs × n_vars = 231646 × 26442
    obs: 'Sample_ID', 'Cell Type', 'Study_name', 'Donor_ID', 'Diagnosis', 'Age', 'Region code', 'Fraction', 'Sex', 'Library_Preparation_Protocol', 'batch', 'Age_group', 'Location', 'Cell States', 'Cell States GCA', 'Chem', 'Layer', 'Cell States Kong', 'dataset', 'n_genes_by_counts', 'total_counts', 'total_counts_mito', 'pct_counts_mito', 'total_counts_ribo', 'pct_counts_ribo', 'Cell_ID', '_scvi_batch', '_scvi_labels'
    var: 'feature_types-0-0-0', 'gene_name-1-0-0', 'gene_id-0-0', 'GENE-1-0'

In [10]:
# Save raw data
adata.raw = adata

In [11]:
sc.pp.filter_genes(adata, min_counts = 1)
sc.pp.filter_genes(adata, min_cells = 3)

sc.pp.filter_cells(adata, min_genes = 50)
sc.pp.filter_cells(adata, min_counts = 3)

filtered out 378 genes that are detected in less than 1 counts
filtered out 619 genes that are detected in less than 3 cells


In [12]:
X_is_raw(adata)

True

### Calculate HVGs

In [13]:
adata.layers['counts'] = adata.X.copy()

In [16]:
adata.obs['Fraction'].value_counts()


Fraction
SC           151069
SC-EPCAMP     54327
SC-EPCAMN     26250
Name: count, dtype: int64

In [14]:
adata.obs

,Sample_ID,Cell Type,Study_name,Donor_ID,Diagnosis,Age,Region code,Fraction,Sex,Library_Preparation_Protocol,...,total_counts,total_counts_mito,pct_counts_mito,total_counts_ribo,pct_counts_ribo,Cell_ID,_scvi_batch,_scvi_labels,n_genes,n_counts
cell_id,,,,,,,,,,,,,,,,,,,,,
AAACCTGAGCATCATC-1-Human_colon_16S8159182,F72-FPIL-0-SC-1,Epithelial,"Elmentaite, 2021",F72,Fetal Healthy,16Wk,FPIL,SC,Female,10x 5' v1,...,10112.0,308.0,3.045886,3050.0,30.162182,AAACCTGAGCATCATC-1-Human_colon_16S8159182,0,2,2726,10445.0
AAACCTGAGCGATCCC-1-Human_colon_16S8159182,F72-FPIL-0-SC-1,Epithelial,"Elmentaite, 2021",F72,Fetal Healthy,16Wk,FPIL,SC,Female,10x 5' v1,...,25347.0,1247.0,4.919714,9091.0,35.866177,AAACCTGAGCGATCCC-1-Human_colon_16S8159182,0,2,4069,26046.0
AAACCTGAGCGTTTAC-1-Human_colon_16S8159182,F72-FPIL-0-SC-1,Epithelial,"Elmentaite, 2021",F72,Fetal Healthy,16Wk,FPIL,SC,Female,10x 5' v1,...,2388.0,209.0,8.752093,507.0,21.231155,AAACCTGAGCGTTTAC-1-Human_colon_16S8159182,0,2,1118,2452.0
AAACCTGAGGATGTAT-1-Human_colon_16S8159182,F72-FPIL-0-SC-1,T cells,"Elmentaite, 2021",F72,Fetal Healthy,16Wk,FPIL,SC,Female,10x 5' v1,...,1866.0,91.0,4.876742,448.0,24.008575,AAACCTGAGGATGTAT-1-Human_colon_16S8159182,0,9,865,1912.0
AAACCTGCAAATTGCC-1-Human_colon_16S8159182,F72-FPIL-0-SC-1,Endothelial,"Elmentaite, 2021",F72,Fetal Healthy,16Wk,FPIL,SC,Female,10x 5' v1,...,2705.0,284.0,10.499076,368.0,13.604437,AAACCTGCAAATTGCC-1-Human_colon_16S8159182,0,1,1424,2797.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTTGTCAGTACATGTC-1-4918STDY7718977,BRC2134_10Wk_FLI_SC-EPCAMP,Epithelial,"Elmentaite, 2021",BRC2134,Fetal Healthy,10Wk,FLI,SC-EPCAMP,Male,10x 3' v1,...,1229.0,54.0,4.393816,388.0,31.570383,TTTGTCAGTACATGTC-1-4918STDY7718977,0,2,712,1282.0
TTTGTCATCCAAGCCG-1-4918STDY7718977,BRC2134_10Wk_FLI_SC-EPCAMP,Mesenchymal,"Elmentaite, 2021",BRC2134,Fetal Healthy,10Wk,FLI,SC-EPCAMP,Male,10x 3' v1,...,6254.0,99.0,1.582987,2664.0,42.596741,TTTGTCATCCAAGCCG-1-4918STDY7718977,0,3,1982,6416.0
TTTGTCATCCATGCTC-1-4918STDY7718977,BRC2134_10Wk_FLI_SC-EPCAMP,Mesenchymal,"Elmentaite, 2021",BRC2134,Fetal Healthy,10Wk,FLI,SC-EPCAMP,Male,10x 3' v1,...,3272.0,44.0,1.344743,1044.0,31.907091,TTTGTCATCCATGCTC-1-4918STDY7718977,0,3,1483,3389.0


In [17]:
sc.pp.highly_variable_genes(
    adata,
    flavor = "seurat_v3",
    n_top_genes = 5000,
    layer = "counts",
    batch_key = "Library_Preparation_Protocol",
    subset = True,
    span = 1
)
adata

If you pass `n_top_genes`, all cutoffs are ignored.
extracting highly variable genes
--> added
    'highly_variable', boolean vector (adata.var)
    'highly_variable_rank', float vector (adata.var)
    'means', float vector (adata.var)
    'variances', float vector (adata.var)
    'variances_norm', float vector (adata.var)


AnnData object with n_obs × n_vars = 231646 × 5000
    obs: 'Sample_ID', 'Cell Type', 'Study_name', 'Donor_ID', 'Diagnosis', 'Age', 'Region code', 'Fraction', 'Sex', 'Library_Preparation_Protocol', 'batch', 'Age_group', 'Location', 'Cell States', 'Cell States GCA', 'Chem', 'Layer', 'Cell States Kong', 'dataset', 'n_genes_by_counts', 'total_counts', 'total_counts_mito', 'pct_counts_mito', 'total_counts_ribo', 'pct_counts_ribo', 'Cell_ID', '_scvi_batch', '_scvi_labels', 'n_genes', 'n_counts'
    var: 'feature_types-0-0-0', 'gene_name-1-0-0', 'gene_id-0-0', 'GENE-1-0', 'n_counts', 'n_cells', 'highly_variable', 'highly_variable_rank', 'means', 'variances', 'variances_norm', 'highly_variable_nbatches'
    uns: 'hvg'
    layers: 'counts'

### Data integration with `scVI`

In [18]:
scvi.model.SCVI.setup_anndata(
    adata,
    layer = "counts",
    categorical_covariate_keys = ["Sample_ID"],
    continuous_covariate_keys = ["n_genes", "n_counts"]
)

An NVIDIA GPU may be present on this machine, but a CUDA-enabled jaxlib is not installed. Falling back to cpu.


In [19]:
model = scvi.model.SCVI(adata, n_layers = 3, n_latent = 50, gene_likelihood = "nb", dispersion = 'gene-batch')
model

SCVI Model with the following params: 
n_hidden: 128, n_latent: 50, n_layers: 3, dropout_rate: 0.1, dispersion: gene-batch, gene_likelihood: nb, 
latent_distribution: normal
Training status: Not Trained
Model's adata is minified?: False

In [20]:
model.train()

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


Epoch 35/35: 100%|██████████████████████████████████████████| 35/35 [05:56<00:00,  9.43s/it, v_num=1, train_loss_step=1.34e+3, train_loss_epoch=1.27e+3]

`Trainer.fit` stopped: `max_epochs=35` reached.


Epoch 35/35: 100%|██████████████████████████████████████████| 35/35 [05:56<00:00, 10.18s/it, v_num=1, train_loss_step=1.34e+3, train_loss_epoch=1.27e+3]


In [21]:
latent = model.get_latent_representation()
adata.obsm["X_scVI"] = latent

In [22]:
sc.pp.neighbors(adata, use_rep = "X_scVI", n_neighbors = 50, metric = 'minkowski')
sc.tl.umap(adata, min_dist = 0.3, spread = 1, random_state = 1712)

computing neighbors
    finished: added to `.uns['neighbors']`
    `.obsp['distances']`, distances for each pair of neighbors
    `.obsp['connectivities']`, weighted adjacency matrix (0:01:15)
computing UMAP
    finished: added
    'X_umap', UMAP coordinates (adata.obsm) (0:02:01)


In [23]:
adata.write_h5ad('/home/amaguza/data/Processed_data/Gut_data/Stem_cells/Fetal_cells_scvi.h5ad')

In [7]:
adata = sc.read_h5ad('/home/amaguza/data/Processed_data/Gut_data/Stem_cells/Fetal_cells_scvi.h5ad')
adata

AnnData object with n_obs × n_vars = 231646 × 5000
    obs: 'Sample_ID', 'Cell Type', 'Study_name', 'Donor_ID', 'Diagnosis', 'Age', 'Region code', 'Fraction', 'Sex', 'Library_Preparation_Protocol', 'batch', 'Age_group', 'Location', 'Cell States', 'Cell States GCA', 'Chem', 'Layer', 'Cell States Kong', 'dataset', 'n_genes_by_counts', 'total_counts', 'total_counts_mito', 'pct_counts_mito', 'total_counts_ribo', 'pct_counts_ribo', 'Cell_ID', '_scvi_batch', '_scvi_labels', 'n_genes', 'n_counts'
    var: 'feature_types-0-0-0', 'gene_name-1-0-0', 'gene_id-0-0', 'GENE-1-0', 'n_counts', 'n_cells', 'highly_variable', 'highly_variable_rank', 'means', 'variances', 'variances_norm', 'highly_variable_nbatches'
    uns: '_scvi_manager_uuid', '_scvi_uuid', 'hvg', 'neighbors', 'umap'
    obsm: 'X_scVI', 'X_umap', '_scvi_extra_categorical_covs', '_scvi_extra_continuous_covs'
    layers: 'counts'
    obsp: 'connectivities', 'distances'

In [ ]:
sc.pl.umap(adata, frameon = False, color = ['Cell States'], color_map='magma_r', size = 1, legend_fontsize = 5, ncols = 4)

In [8]:
df = adata.obs['Cell States'].value_counts()

In [9]:
# Make a column adata.obs['ASS1+_SLC40A1+_SC'], and put true there if adata.obs['Cell States'] == 'ASS1+_SLC40A1+_SC'
adata.obs['ASS1+_SLC40A1+_SC'] = adata.obs['Cell States'] == 'ASS1+_SLC40A1+_SC'
adata.obs['FXYD3+_CKB+_SC'] = adata.obs['Cell States'] == 'FXYD3+_CKB+_SC'
adata.obs['RPS10+_RPS17+_SC'] = adata.obs['Cell States'] == 'RPS10+_RPS17+_SC'

# Change adata.obs['ASS1+_SLC40A1+_SC'] to categorical
adata.obs['ASS1+_SLC40A1+_SC'] = adata.obs['ASS1+_SLC40A1+_SC'].astype('category')
adata.obs['FXYD3+_CKB+_SC'] = adata.obs['FXYD3+_CKB+_SC'].astype('category')
adata.obs['RPS10+_RPS17+_SC'] = adata.obs['RPS10+_RPS17+_SC'].astype('category')

In [ ]:
sc.pl.umap(adata, frameon = False, color = ['ASS1+_SLC40A1+_SC', 'RPS10+_RPS17+_SC', 'FXYD3+_CKB+_SC'], size = 1, legend_fontsize = 5, ncols = 4)

In [ ]:
# Plot umap with leiden clusters
sc.set_figure_params(dpi=300)
sc.pl.umap(adata, frameon = False, color = ['Cell Type', 'Donor_ID', 'Age', 'Age_group', 'Library_Preparation_Protocol', 'Sex', 'n_genes_by_counts', 'total_counts', 'pct_counts_mito', 'pct_counts_ribo'], size = 1, legend_fontsize = 5, ncols = 3, color_map='magma')

In [ ]:
# Plot umap with leiden clusters
sc.set_figure_params(dpi=300)
sc.pl.umap(adata, frameon = False, color = ['ASS1+_SLC40A1+_SC', 'MTRNR2L12', 'ASS1', 'CCL25', 'MT-ATP8', 'CYP2W1', 'PHGR1', 'SLC40A1', 'SAT1',
                                            'OAT', 'ANPEP', 'XIST', 'KRT18', 'IFITM1', 'RPS26', 'NFKBIA', 'NENF', 'FAM13A', 'SQSTM1',
                                            'JUN', 'CYSTM1', 'MT-ND4L', 'PCBP1', 'SPINK1'], size = 1, legend_fontsize = 5, ncols = 4, color_map='magma_r')

In [ ]:
# Plot umap with leiden clusters
sc.set_figure_params(dpi=300)
sc.pl.umap(adata, frameon = False, color = ['RPS10+_RPS17+_SC', "RPS10", "RPS17", "RPS4Y1", "NDUFA13", "MDK", "NDUFC2", "PSMA2", "NDUFA11", "NME1", "CSNK2B",
                                             "PSMA1", "RPL26", "NDUFB8", "APOA1", "MINOS1", "MALAT1", "ARL6IP1", "ATP5PO", "AFP", "GCSH", "MRPL23", 
                                             "EEF1E1", "NAP1L1", "TMBIM4"], size = 1, legend_fontsize = 5, ncols = 4, color_map='magma_r')

In [ ]:
# Plot umap with leiden clusters
sc.set_figure_params(dpi=300)
sc.pl.umap(adata, frameon = False, color = ['FXYD3+_CKB+_SC', "FXYD3", "CKB", "KRT19", "AKAP7", "S100A6", "ITM2C", "FOS", "RHOB", "PPP1R15A",
                                            "C19orf33", "CST3", "LCN15", "EGR1", "TFF3", "DUSP1", "UBC", "C15orf48", "CA12", "H3F3B", 
                                            "FABP1", "CTGF", "PPDPF", "IGF2-1", "BTG2", "MYL6", "LGALS4", "JUN"], 
                                            size = 1, legend_fontsize = 5, ncols = 4, color_map='magma_r')

In [18]:
stem_cells_markers = ['LGR5', 'BMI1', 'GJA1', 'TACSTD2', 'SOX2', 'NANOG', 'LY6H', 'SPP1', 'DCLK1',
                                            'CD44', 'DCLK1', 'TERT', 'ALCAM', 'ASCL2', 'BMPR1A', 'EPHB2']
# perform gene enrichment analysis on the stem cells markers
sc.tl.score_genes(adata, stem_cells_markers, score_name = "Stem_cells_markers_score")

computing score 'Stem_cells_markers_score'
    finished: added
    'Stem_cells_markers_score', score of gene set (adata.obs).
    499 total control genes are used. (0:00:03)


In [ ]:
# Plot umap with leiden clusters
sc.set_figure_params(dpi=300)
sc.pl.umap(adata, frameon = False, color = ['Cell Type','ASS1+_SLC40A1+_SC', 'RPS10+_RPS17+_SC', 'FXYD3+_CKB+_SC', 'LGR5', 'BMI1', 'GJA1', 'TACSTD2', 
                                            'SOX2', 'NANOG', 'LY6H', 'SPP1', 'DCLK1','CD44', 'DCLK1', 'TERT', 'ALCAM', 'ASCL2', 'BMPR1A', 'EPHB2'], 
                                            size = 1, legend_fontsize = 5, ncols = 4, color_map='magma_r')

In [ ]:
sc.set_figure_params(dpi=300)
sc.pl.umap(adata, frameon = False, color = ['LGR5', 'BMI1', 'GJA1', 'TACSTD2', 'SOX2', 'NANOG', 'LY6H', 'SPP1', 'DCLK1', 'CD44', 'DCLK1', 'TERT', 'ALCAM', 'ASCL2', 'BMPR1A', 'EPHB2', 'ACAD10', 
                      'ACVR1C', 'ARSE', 'ASCL2', 'ATP10B', 'C16orf89', 'C6orf136', 'CDCA7', 'CFTR', 'CHMP4C', 'CHP2', 'CLDN15', 'CLDN18', 'CLDN2', 'CPA6', 'DAPK2', 'DDC', 
                      'EFNA3', 'EPHB2', 'EVPL', 'F2RL1', 'FBLN2', 'FOXD2-AS1', 'GATA6-AS1', 'GDF15', 'GJB1', 'GJB2', 'GOLT1A', 'GPX2', 'HNF1A', 'HSD17B2', 'ITPKC', 
                      'LEFTY1', 'LIPG', 'MGST1', 'MSI1', 'MYOM3', 'NOX1', 'OLFM4', 'PCSK9', 'PDZD3', 'PHLDA1', 'PKP2', 'PLAGL2', 'PLEKHH1', 
                      'PPP1R1B', 'PTGDR', 'PTK7', 'RGMB', 'RNF157', 'RNF186', 'SFN', 'SLC27A2', 'SLC38A4', 'SLPI', 'SULT1B1', 'TAF4B', 'TANC1', 'TMEM171', 'TSPAN8', 
                      'URB1-AS1', 'ZBED9', 'ZNF296'], 
                                            size = 1, legend_fontsize = 5, ncols = 10, color_map='magma_r')

In [ ]:
stem_cells_markers = ['LGR5', 'BMI1', 'GJA1', 'TACSTD2', 'SOX2', 'NANOG', 'LY6H', 'SPP1', 'DCLK1', 'CD44', 'DCLK1', 'TERT', 'ALCAM', 'ASCL2', 'BMPR1A', 'EPHB2', 'ACAD10', 
                      'ACVR1C', 'ARSE', 'ASCL2', 'ATP10B', 'C16orf89', 'C6orf136', 'CDCA7', 'CFTR', 'CHMP4C', 'CHP2', 'CLDN15', 'CLDN18', 'CLDN2', 'CPA6', 'DAPK2', 'DDC', 
                      'EFNA3', 'EPHB2', 'EVPL', 'F2RL1', 'FBLN2', 'FOXD2-AS1', 'GATA6-AS1', 'GDF15', 'GJB1', 'GJB2', 'GOLT1A', 'GPX2', 'HNF1A', 'HSD17B2', 'ITPKC', 
                      'LEFTY1', 'LIPG', 'MGST1', 'MSI1', 'MYOM3', 'NOX1', 'OLFM4', 'PCSK9', 'PDZD3', 'PHLDA1', 'PKP2', 'PLAGL2', 'PLEKHH1', 
                      'PPP1R1B', 'PTGDR', 'PTK7', 'RGMB', 'RNF157', 'RNF186', 'SFN', 'SLC27A2', 'SLC38A4', 'SLPI', 'SULT1B1', 'TAF4B', 'TANC1', 'TMEM171', 'TSPAN8', 
                      'URB1-AS1', 'ZBED9', 'ZNF296']

sc.pl.dotplot(adata, stem_cells_markers, groupby='Cell States')

In [ ]:
stem_cells_markers = ['LGR5', 'BMI1', 'TACSTD2', 'SOX2', 'NANOG', 'LY6H', 'DCLK1', 'CD44', 'DCLK1', 'TERT', 'ALCAM', 'ASCL2', 'BMPR1A', 'EPHB2', 'ACAD10', 
                      'ACVR1C', 'ARSE', 'ASCL2', 'ATP10B', 'C16orf89', 'C6orf136', 'CDCA7', 'CFTR', 'CHMP4C', 'CHP2', 'CLDN15', 'CLDN18', 'CLDN2', 'CPA6', 'DAPK2', 
                      'EFNA3', 'EPHB2', 'EVPL', 'F2RL1', 'FOXD2-AS1', 'GATA6-AS1', 'GDF15', 'GJB1', 'GJB2', 'GOLT1A', 'GPX2', 'HNF1A', 'HSD17B2', 'ITPKC', 
                      'LEFTY1', 'LIPG', 'MGST1', 'MSI1', 'MYOM3', 'NOX1', 'OLFM4', 'PCSK9', 'PDZD3', 'PHLDA1', 'PKP2', 'PLAGL2', 'PLEKHH1', 
                      'PPP1R1B', 'PTGDR', 'PTK7', 'RGMB', 'RNF157', 'RNF186', 'SFN', 'SLC27A2', 'SLC38A4', 'SLPI', 'SULT1B1', 'TAF4B', 'TANC1', 'TMEM171', 
                      'URB1-AS1', 'ZBED9', 'ZNF296']

sc.pl.dotplot(adata, stem_cells_markers, groupby='Cell States')

In [22]:
stem_cells_markers = ['LGR5', 'BMI1', 'GJA1', 'TACSTD2', 'SOX2', 'NANOG', 'LY6H', 'SPP1', 'DCLK1', 'CD44', 'DCLK1', 'TERT', 'ALCAM', 'ASCL2', 'BMPR1A', 'EPHB2', 'ACAD10', 
                      'ACVR1C', 'ARSE', 'ASCL2', 'ATP10B', 'C16orf89', 'C6orf136', 'CDCA7', 'CFTR', 'CHMP4C', 'CHP2', 'CLDN15', 'CLDN18', 'CLDN2', 'CPA6', 'DAPK2', 'DDC', 
                      'EFNA3', 'EPHB2', 'EVPL', 'F2RL1', 'FBLN2', 'FOXD2-AS1', 'GATA6-AS1', 'GDF15', 'GJB1', 'GJB2', 'GOLT1A', 'GPX2', 'HNF1A', 'HSD17B2', 'ITPKC', 
                      'LEFTY1', 'LIPG', 'MGST1', 'MSI1', 'MYOM3', 'NOX1', 'OLFM4', 'PCSK9', 'PDZD3', 'PHLDA1', 'PKP2', 'PLAGL2', 'PLEKHH1', 
                      'PPP1R1B', 'PTGDR', 'PTK7', 'RGMB', 'RNF157', 'RNF186', 'SFN', 'SLC27A2', 'SLC38A4', 'SLPI', 'SULT1B1', 'TAF4B', 'TANC1', 'TMEM171', 'TSPAN8', 
                      'URB1-AS1', 'ZBED9', 'ZNF296']

# perform gene enrichment analysis on the stem cells markers
sc.tl.score_genes(adata, stem_cells_markers, score_name = "Stem_cells_markers_score")

computing score 'Stem_cells_markers_score'
    finished: added
    'Stem_cells_markers_score', score of gene set (adata.obs).
    796 total control genes are used. (0:00:03)
